<h2 style="font-size:24pt"> Proyecto DESI</h2>

<h2 style="font-size:24pt"> Julio 11, 2025</h2>

<p style="font-size:16pt">
Classification groups: filaments with real data, knots with real data, and voids with random data.

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.table import Table
import pandas as pd
import re
from astropy.io import ascii

In [3]:
connections = {}
for j in range(100):
    file = f"connections_rosette/LRG_connection_rosette_{0}_rand_{j}.ecsv"
    table = Table.read(file, format="ascii.ecsv")
    connection = table[['TARGETID','type', f'class_{j}','neighbor_ids_data','neighbor_ids_rand']]
    connections[f'rosetta_{i}_random_{j}'] = connection

In [ ]:
def func_connections(df, classification, type_data, i, j):
    id_to_type = dict(zip(df['TARGETID'], df['type']))
    id_to_class = dict(zip(df['TARGETID'], df[f'class_{j}']))

    # Filtrar solo los que son tipo 'data' y clase correspondiente
    mask_data_class = (df['type'] == 'data') & (df[f'class_{j}'] == classification)
    df_data_class = df[mask_data_class].copy().to_pandas()

    # Diccionario de conexiones
    data_class_connections = {}

    for index, row in df_data_class.iterrows():
        center_id = row['TARGETID']
        neighbor_str = row.get(f'neighbor_ids_{type_data}', '')

        # Omitir si está vacío, nulo o contiene '--'
        if pd.isna(neighbor_str) or neighbor_str.strip() in {'--', ''}:
            continue

        # Convertir vecinos a enteros válidos
        neighbors = [int(x) for x in neighbor_str.split(',') if x.strip().isdigit()]

        # Filtrar vecinos que también sean del tipo y clase deseados
        valid_neighbors = [
            n for n in neighbors
            if id_to_type.get(n) == type_data and id_to_class.get(n) == classification
        ]
        data_class_connections[center_id] = valid_neighbors

    # Crear DataFrame con conexiones
    rows = [(center, neighbor) for center, neighbors in data_class_connections.items() for neighbor in neighbors]
    df_connections = pd.DataFrame(rows, columns=["TARGETID", f"TARGETID_{classification}"])
    
    # Guardar como tabla ECSV
    table_connections = Table.from_pandas(df_connections)
    filename = f"{classification}_rosette/LRG_{classification}_rosette_{i}_random_{j}.ecsv"
    ascii.write(table_connections, filename, format='ecsv', overwrite=True)


In [19]:
%%time
for j in range(100):
    func_connections(connections[f'rosetta_0_random_{j}'],'filament','data',i,j)

CPU times: total: 49.8 s
Wall time: 59.9 s


In [38]:
%%time
for i in range(100):
    func_connections(connections[f'rosetta_0_random_{j}'],'knot','data',i,j)

CPU times: total: 4.98 s
Wall time: 5.2 s


In [39]:
%%time
for i in range(100):
    func_connections(connections[f'rosetta_0_random_{j}'],'void','rand',i,j)

CPU times: total: 6.28 s
Wall time: 6.67 s
